# Parallel GST using MPI
The purpose of this tutorial is to demonstrate how to compute GST estimates in parallel (using multiple CPUs or "processors").  The core PyGSTi computational routines are written to take advantage of multiple processors via the MPI communication framework, and so one must have a version of MPI and the `mpi4py` python package installed in order use run pyGSTi calculations in parallel.  

Since `mpi4py` doesn't play nicely with Jupyter notebooks, this tutorial is a bit more clunky than the others.  In it, we will create a standalone Python script that imports `mpi4py` and execute it.

We will use as an example the same "standard" single-qubit gate set of the first tutorial.  We'll first create a dataset, and then a script to be run in parallel which loads the data.  The creation of a simulated data is performed in the same way as the first tutorial.   Since *random* numbers are generated and used as simulated counts within the call to `generate_fake_data`, it is important that this is *not* done in a parallel environment, or different CPUs may get different data sets.  (This isn't an issue in the typical situation when the data is obtained experimentally.)

In [1]:
#Import pyGSTi and the "stardard 1-qubit quantities for a gateset with X(pi/2), Y(pi/2), and idle gates"
import pygsti
from pygsti.construction import std1Q_XYI

#Create a data set
gs_target = std1Q_XYI.gs_target
fiducials = std1Q_XYI.fiducials
germs = std1Q_XYI.germs
maxLengths = [1,2,4,8,16,32]

gs_datagen = gs_target.depolarize(gate_noise=0.1, spam_noise=0.001)
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(gs_target.gates.keys(), fiducials, fiducials, germs, maxLengths)
ds = pygsti.construction.generate_fake_data(gs_datagen, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)
pygsti.io.write_dataset("example_files/mpi_example_dataset.txt", ds)

Next, we'll write a Python script that will load in the just-created `DataSet`, run GST on it, and write the output to a file.  The only major difference between the contents of this script and previous examples is that the script imports `mpi4py` and passes a MPI comm object (`comm`) to the `do_long_sequence_gst` function.  Since parallel computing is best used for computationaly intensive GST calculations, we also demonstrate how to set a per-processor memory limit to tell pyGSTi to partition its computations so as to not exceed this memory usage.  Lastly, note the use of the `gaugeOptParams` argument of `do_long_sequence_gst`, which can be used to weight different gate set members differently during gauge optimization.

In [2]:
mpiScript = """
import time
import pygsti
from pygsti.construction import std1Q_XYI

#get MPI comm
from mpi4py import MPI
comm = MPI.COMM_WORLD

print("Rank %d started" % comm.Get_rank())

#define target gateset, fiducials, and germs as before
gs_target = std1Q_XYI.gs_target
fiducials = std1Q_XYI.fiducials
germs = std1Q_XYI.germs
maxLengths = [1,2,4,8,16,32]

#tell gauge optimization to weight the gate matrix
# elements 100x more heavily than the SPAM vector elements, and
# to specifically weight the Gx gate twice as heavily as the other
# gates.
goParams = {'itemWeights':{'spam': 0.01, 'gates': 1.0, 'Gx': 2.0} }

#Specify a per-core memory limit (useful for larger GST calculations)
memLim = 2.1*(1024)**3  # 2.1 GB

#Perform TP-constrained GST
gs_target.set_all_parameterizations("TP")
    
#load the dataset
ds = pygsti.io.load_dataset("example_files/mpi_example_dataset.txt")

start = time.time()
results = pygsti.do_long_sequence_gst(ds, gs_target, fiducials, fiducials,
                                      germs, maxLengths,memLimit=memLim,
                                      gaugeOptParams=goParams, comm=comm,
                                      verbosity=2)
end = time.time()
print("Rank %d finished in %.1fs" % (comm.Get_rank(), end-start))
if comm.Get_rank() == 0:
    import pickle
    pickle.dump(results, open("example_files/mpi_example_results.pkl","wb"))
"""
with open("example_files/mpi_example_script.py","w") as f:
    f.write(mpiScript)

Next, we run the script with 3 processors using `mpiexec`.  The `mpiexec` executable should have been installed with your MPI distribution -- if it doesn't exist, try replacing `mpiexec` with `mpirun`.

In [3]:
! mpiexec -n 3 python3 "example_files/mpi_example_script.py"

Rank 1 started
Rank 2 started
Rank 0 started
--- Gate Sequence Creation ---
   1702 sequences created
   Dataset has 1702 entries: 1702 utilized, 0 requested sequences were missing
--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  4.245058951635965
  1.1585845663892917
  0.9677945789630839
  0.9223871580426515
  0.07048465733093594
  0.0213024962973271
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  4.242640687119285
  1.4142135623730954
  1.4142135623730947
  1.4142135623730945
  3.1723744950054595e-16
  1.0852733691121267e-16
  
--- Iterative MLGST: Iter 1 of 6  92 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Memory limit = 2.10GB
  Cur, Persist, Gather = 0.11, 0.00, 0.21 GB
  Finding num_nongauge_params is too expensive: using total params.
  Sum of Chi^2 = 86.4516 (92 data params - 43 model params = expected mean of 49; p-value = 0.000769292)
  Completed in 0.3s
  2*Delta(log(L)) = 86.7954
  Iteration 1 took 0.4s
  
--- Iterati

Notice in the above that output within `do_long_sequence_gst` is not duplicated (only the first processor outputs to stdout) so that the output looks identical to running on a single processor.  Finally, we just need to read the pickled `Results` object from file and proceed with any post-processing analysis.  In this case, we'll just create a  report. 

In [4]:
import pickle
results = pickle.load(open("example_files/mpi_example_results.pkl","rb"))
pygsti.report.create_standard_report(results, "example_files/mpi_example_brief",
                                    title="MPI Example Report", verbosity=2, auto_open=True)

*** Creating workspace ***
*** Generating switchboard ***
Found standard clifford compilation from std1Q_XYI
*** Generating tables ***
  targetSpamBriefTable                          took 0.036339 seconds
  targetGatesBoxTable                           took 0.042545 seconds
  datasetOverviewTable                          took 0.074418 seconds
  bestGatesetSpamParametersTable                took 0.000744 seconds
  bestGatesetSpamBriefTable                     took 0.044855 seconds
  bestGatesetSpamVsTargetTable                  took 0.10052 seconds
  bestGatesetGaugeOptParamsTable                took 0.000615 seconds
  bestGatesetGatesBoxTable                      took 0.043628 seconds
  bestGatesetChoiEvalTable                      took 0.064883 seconds
  bestGatesetDecompTable                        took 0.053313 seconds
  bestGatesetEvalTable                          took 0.004118 seconds
  bestGermsEvalTable                            took 0.02304 seconds
  bestGatesetVsTargetTable 